In [2]:
import asyncio
import os
import tempfile
from pathlib import Path
import nest_asyncio
from seeact.agent import SeeActAgent

# Enable nest_asyncio for running async code in notebooks
nest_asyncio.apply()

async def run_agent(api_key: str, temp_dir: str = None):
    """
    运行 SeeAct agent 并包含监控和错误处理
    
    Args:
        api_key: Gemini API key
        temp_dir: 可选的临时目录路径
    """
    try:
        # 设置 API key
        os.environ["GEMINI_API_KEY"] = api_key
        
        # 使用系统临时目录或指定目录
        working_dir = temp_dir or tempfile.gettempdir()
        save_dir = Path(working_dir) / "seeact_agent_files"
        save_dir.mkdir(parents=True, exist_ok=True)
        
        # 初始化 agent，使用更安全的配置
        agent = SeeActAgent(
            model="gemini-1.5-pro-latest",
            save_file_dir=str(save_dir),
            default_task="Find the pdf of the paper 'GPT-4V(ision) is a Generalist Web Agent, if Grounded'",
            default_website="https://www.google.com/",
            headless=True,
            temperature=0.7,  # 降低温度以获得更稳定的输出
            rate_limit=1,  # 限制请求速率
            grounding_strategy="text_choice_som",  # 使用更稳定的定位策略
            crawler_mode=False,  # 禁用爬虫模式以提高安全性
            max_auto_op=5  # 限制自动操作次数
        )
        
        print("启动 agent...")
        await agent.start()
        
        while not agent.complete_flag:
            try:
                prediction_dict = await agent.predict()
                if prediction_dict:
                    # 监控和确认机制
                    print(f"\n预测的操作: {prediction_dict}")
                    confirm = input("\n是否执行此操作? [Y/n/i/e] (Y=是, n=否, i=人工干预, e=终止): ").lower()
                    
                    if confirm == 'e':
                        print("终止执行...")
                        break
                    elif confirm == 'i':
                        message = input("请输入人工操作说明: ")
                        # 这里可以添加处理人工操作的逻辑
                        continue
                    elif confirm == 'n':
                        print("跳过此操作...")
                        continue
                    
                    # 执行操作
                    if prediction_dict.get('action') == 'CLICK':
                        try:
                            await agent.execute(prediction_dict)
                        except Exception as e:
                            if "intercepted" in str(e) or "timeout" in str(e):
                                print("点击操作失败，尝试使用 PRESS ENTER")
                                await agent.execute({'action': 'PRESS ENTER', 'value': None})
                            else:
                                raise
                    else:
                        await agent.execute(prediction_dict)
                
                # 添加延迟避免过快操作
                await asyncio.sleep(0.5)
                
            except Exception as e:
                print(f"操作执行错误: {e}")
                # 出错时等待更长时间
                await asyncio.sleep(1)
                
    except Exception as e:
        print(f"运行时错误: {e}")
        raise
        
    finally:
        # 清理资源
        if 'agent' in locals():
            try:
                await agent.stop()
            except:
                pass

# 使用示例
async def main():
    api_key 
    await run_agent(api_key)

if __name__ == "__main__":
    asyncio.run(main())

Mark page script error [Errno 2] No such file or directory: '/Users/xiangzhang/Library/Python/3.9/lib/python/site-packages/seeact/mark_page.js'
/var/folders/n4/274598s91l591znzcg56fhhw0000gn/T/seeact_agent_files/20241022_073221/screenshots/screen_6.png
Step - 6
----------
Action Generation ➡️
TASK: Find the pdf of the paper 'GPT-4V(ision) is a Generalist Web Agent, if Grounded'
Previous:
[textarea role="combobox"] aria-label="Search" name="q" title="Search" -> TYPE: GPT-4V(ision) is a Generalist Web Agent, if Grounded
Failed to perform CLICK on aria-label="Google Search" name="btnK" value="Google Search" with value 'None': Locator.click: Timeout 2000ms exceeded.
Call log:
waiting for locator("input").nth(2)
  -   locator resolved to <input name="btnK" tabindex="0" role="button" type="submit" class="gNO89b" value="Google Search" aria-label="Google Search" data-ved="0ahUKEwjYu-b78aGJAxXgAHkGHTLaGRIQ4dUDCBM"/>
  - attempting click action
  -   waiting for element to be visible, enabled an



Request to litellm:
litellm.completion(model='gemini/gemini-1.5-pro-latest', messages=[{'role': 'system', 'content': 'You are assisting humans doing web navigation tasks step by step. At each stage, you can see the webpage by a screenshot and know the previous actions before the current step decided by yourself that have been executed for this task through recorded history. You need to decide on the first following action to take.\n\nHere are the descriptions of all allowed actions:\n\nNo Value Operations:\n- CLICK: Click on a webpage element using the mouse.\n- HOVER: Move the mouse over a webpage element without clicking.\n- PRESS ENTER: Press the Enter key, typically to submit a form or confirm an input.\n- SCROLL UP: Scroll the webpage upwards by half of the window height.\n- SCROLL DOWN: Scroll the webpage downwards by half of the window height.\n- PRESS HOME: Scroll to the top of the webpage.\n- PRESS END: Scroll to the bottom of the webpage.\n- PRESS PAGEUP: Scroll up by one w

----------
🤖 Action Generation Output 🤖
Target Element: The 'Google Search' button (it's less obstructed than the "I'm Feeling Lucky" button).
Action: CLICK 

----------
Action Grounding ➡️
If none of these elements match your target element or your target action doesn't involve an element, please select AC.
A. (0.035, 0.042) <a">About</a>
B. (0.078, 0.042) <a">Store</a>
C. (0.801, 0.042) <a">Gmail</a>
D. (0.843, 0.042) <a">Images</a>
E. (0.887, 0.042) <a role="button"">aria-label="Google apps"</a>
F. (0.946, 0.042) <a">Sign in</a>
G. (0.831, 0.235) <iframe role="presentation"">name="callout"</iframe>
H. (0.5, 0.242) <img">alt="Google"</img>
I. (0.463, 0.375) <textarea role="combobox"">input value="GPT-4V(ision) is a Generalist Web Agent, if Grounded" parent_node: GPT-4V(ision) is a Generalist Web Agent, if... aria-label="Search" name="q" title="Search"</textarea>
J. (0.5, 0.432) <form role="search"">GPT-4V(ision) is a Generalist Web Agent, if Grounded gpt-4v(vision) is a generalist we

RAW RESPONSE:
response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': 'Target Element: The \'Google Search\' button (it\'s less obstructed than the "I\'m Feeling Lucky" button).\nAction: CLICK \n'}], 'role': 'model'}, 'finish_reason': 1, 'index': 0, 'safety_ratings': [{'category': 9, 'probability': 2, 'blocked': False}, {'category': 8, 'probability': 1, 'blocked': False}, {'category': 7, 'probability': 1, 'blocked': False}, {'category': 10, 'probability': 1, 'blocked': False}], 'token_count': 0, 'grounding_attributions': []}]}),
)


raw model_response: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': 'Target Element: The \'Google Search\' button (it\'s less obstructed than the "I\'m Feeling Lucky" button).\nAction: CLICK \n'}], 'role': 'model'}, 'finish_reason': 1, 'index': 0, 'safe

🤖 Action Grounding Output 🤖
Target Element: The 'Google Search' button (it's less obstructed than the "I'm Feeling Lucky" button).
Action: CLICK
(Reiteration)
Target Element: The 'Google Search' button (it's less obstructed than the "I'm Feeling Lucky" button).
Action: CLICK
(Multichoice Question)
L. (0.44, 0.474) <input role="button" type="submit"">aria-label="Google Search" name="btnK" value="Google Search"</input>

(Final Answer)
ELEMENT: L
ACTION: CLICK
VALUE: None 



...

----------
🤖 Action Generation Output 🤖
TERMINATE: The task is complete as the pdf is open in the browser. 

----------
Action Grounding ➡️
If none of these elements match your target element or your target action doesn't involve an element, please select B.
A. (0.5, 0.5) <embed type="application/pdf"">name="0DE68E7DF8AA0CF07B0D2E83E8AF964D"</embed>
B. None of the other options match the correct element or the action doesn't involve an element.


RAW RESPONSE:
response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': 'TERMINATE: The task is complete as the pdf is open in the browser. \n'}], 'role': 'model'}, 'finish_reason': 1, 'index': 0, 'safety_ratings': [{'category': 9, 'probability': 1, 'blocked': False}, {'category': 8, 'probability': 1, 'blocked': False}, {'category': 7, 'probability': 1, 'blocked': False}, {'category': 10, 'probability': 2, 'blocked': False}], 'token_count': 0, 'grounding_attributions': []}]}),
)


raw model_response: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': 'TERMINATE: The task is complete as the pdf is open in the browser. \n'}], 'role': 'model'}, 'finish_reason': 1, 'index': 0, 'safety_ratings': [{'category': 9, 'probability': 1, 'blocked': False}, {'category': 8, 'probability': 1, 'blocked'

🤖 Action Grounding Output 🤖
(Reiteration)
The task is complete as the pdf is open in the browser.

(Multichoice Question)
B. None of the other options match the correct element or the action doesn't involve an element.

(Final Answer)
ELEMENT: B
ACTION: TERMINATE
VALUE: None 



RAW RESPONSE:
response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': "(Reiteration)\nThe task is complete as the pdf is open in the browser.\n\n(Multichoice Question)\nB. None of the other options match the correct element or the action doesn't involve an element.\n\n(Final Answer)\nELEMENT: B\nACTION: TERMINATE\nVALUE: None \n"}], 'role': 'model'}, 'finish_reason': 1, 'index': 0, 'safety_ratings': [{'category': 9, 'probability': 1, 'blocked': False}, {'category': 8, 'probability': 1, 'blocked': False}, {'category': 7, 'probability': 1, 'blocked': False}, {'category': 10, 'probability': 1, 'blocked': False}], 'token_count': 0, 'grounding_attributions': []}]}),
)


raw model_response: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': "(Reiteration)\nThe task is complete as the pdf is

Task has been marked as complete. Terminating...
Browser context closed.
Agent stopped.
